# Strip Chart Testing

This notebook is used for testing the strip chart mode on  Auxiliary Telescope.

Craig Lage - 02-Jul-21

In [1]:
import sys
import asyncio
import time
import os
import numpy as np
from astropy.time import Time, TimeDelta
from lsst.ts import salobj
from lsst.ts.observatory.control.auxtel.latiss import LATISS

In [2]:
# for tab completion to work in current notebook instance
%config IPCompleter.use_jedi = False

In [3]:
import logging
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG
# Make matplotlib less chatty
logging.getLogger("matplotlib").setLevel(logging.WARNING)

In [5]:
#Start classes
domain = salobj.Domain()
latiss = LATISS(domain)
await latiss.start_task

atcamera: Adding all resources.
atspectrograph: Adding all resources.
atheaderservice: Adding all resources.
atarchiver: Adding all resources.
Read historical data in 0.00 sec
Read 1 history items for RemoteEvent(ATHeaderService, 0, authList)
Read 100 history items for RemoteEvent(ATHeaderService, 0, heartbeat)
Read 100 history items for RemoteEvent(ATHeaderService, 0, largeFileObjectAvailable)
Read 1 history items for RemoteEvent(ATHeaderService, 0, logLevel)
Read 100 history items for RemoteEvent(ATHeaderService, 0, logMessage)
Read 1 history items for RemoteEvent(ATHeaderService, 0, simulationMode)
Read 1 history items for RemoteEvent(ATHeaderService, 0, softwareVersions)
Read 39 history items for RemoteEvent(ATHeaderService, 0, summaryState)
Read historical data in 0.19 sec
Read 10 history items for RemoteEvent(ATSpectrograph, 0, appliedSettingsMatchStart)
Read 1 history items for RemoteEvent(ATSpectrograph, 0, authList)
Read 100 history items for RemoteEvent(ATSpectrograph, 0, dis

[None, None, None, None]

In [ ]:
# enable components
# No need - all enabled
#await latiss.enable({"atspectrograph": "current"})

In [6]:
# Take a bias to make sure everything is working
# Succeeded
await latiss.take_bias(1)

Generating group_id
imagetype: BIAS, skip TCS synchronization.
BIAS 0001 - 0001


array([2021100700116])

logMessage DDS read queue is filling: 16 of 100 elements


Step one - go in and out of calibration.\
This succeeded - I could see the camera substate change.

In [7]:
await latiss.rem.atcamera.cmd_enableCalibration.set_start()

In [8]:
await asyncio.sleep(2.0)

In [9]:
await latiss.rem.atcamera.cmd_disableCalibration.start()

In [10]:
# Take a bias to make sure everything is working
# Succeeded
await latiss.take_bias(1)

Generating group_id
imagetype: BIAS, skip TCS synchronization.
BIAS 0001 - 0001
logMessage DDS read queue is filling: 16 of 100 elements


array([2021100700117])

logMessage DDS read queue is filling: 30 of 100 elements
logMessage DDS read queue is filling: 70 of 100 elements


Step 1.5:
In and out of calibration with a clear.\
Succeeded

In [11]:
await latiss.rem.atcamera.cmd_enableCalibration.set_start()

In [13]:
await latiss.rem.atcamera.cmd_clear.set_start(nClears=2)

In [14]:
await asyncio.sleep(2.0)

In [15]:
await latiss.rem.atcamera.cmd_disableCalibration.start()

In [16]:
# Take a bias to make sure everything is working
# Succeeded
await latiss.take_bias(1)

Generating group_id
imagetype: BIAS, skip TCS synchronization.
BIAS 0001 - 0001
logMessage DDS read queue is filling: 27 of 100 elements


array([2021100700118])

logMessage DDS read queue is filling: 21 of 100 elements


Step 2:
The following commands should take a calibration image\
Try this first without the discardRows()\
This worked.  The shutter opened and closed, and successfully took image 0012.

In [17]:
await latiss.rem.atcamera.cmd_enableCalibration.set_start()

In [18]:
await latiss.rem.atcamera.cmd_clear.set_start(nClears=2)

In [19]:
keyValueMap = f"groupId: {Time.now().isot},imageType: ENGTEST" 

In [20]:
print(keyValueMap)

groupId: 2021-10-07T18:38:51.705,imageType: ENGTEST


In [21]:
latiss.rem.atcamera.cmd_startImage.set(shutter=True, keyValueMap=keyValueMap, timeout=10.0)

True

In [22]:
await latiss.rem.atcamera.cmd_startImage.start()

In [23]:
await asyncio.sleep(2.0)

In [ ]:
await latiss.rem.atcamera.cmd_endImage.start()
# Worked this time

In [24]:
await latiss.rem.atcamera.cmd_disableCalibration.start()

Step 2A:
The following commands should take a calibration image\
Try this first without the discardRows()\
All in one cell so we get a real 2 second image

In [27]:
await latiss.rem.atcamera.cmd_enableCalibration.set_start()
await latiss.rem.atcamera.cmd_clear.set_start(nClears=2)
keyValueMap = f"groupId: {Time.now().isot},imageType: ENGTEST"
print(keyValueMap)
latiss.rem.atcamera.cmd_startImage.set(shutter=True, keyValueMap=keyValueMap, timeout=10.0)
await latiss.rem.atcamera.cmd_startImage.start()
await asyncio.sleep(2.0)
await latiss.rem.atcamera.cmd_endImage.start()
await latiss.rem.atcamera.cmd_disableCalibration.start()


groupId: 2021-10-07T18:46:18.223,imageType: ENGTEST


In [26]:
# Take a bias to make sure everything is working
# Succeeded.
await latiss.take_bias(1)

Generating group_id
imagetype: BIAS, skip TCS synchronization.
BIAS 0001 - 0001


array([2021100700121])

Step 2B:
The following commands should take a calibration image\
with the discardRows()\
All in one cell so we get a real 2 second image

In [28]:
await latiss.rem.atcamera.cmd_enableCalibration.set_start()
await latiss.rem.atcamera.cmd_clear.set_start(nClears=2)
keyValueMap = f"groupId: {Time.now().isot},imageType: ENGTEST"
print(keyValueMap)
latiss.rem.atcamera.cmd_startImage.set(shutter=True, keyValueMap=keyValueMap, timeout=10.0)
await latiss.rem.atcamera.cmd_startImage.start()
await asyncio.sleep(1.0)
latiss.rem.atcamera.cmd_discardRows.set(nRows=500)
await latiss.rem.atcamera.cmd_discardRows.start()
await asyncio.sleep(1.0)
await latiss.rem.atcamera.cmd_endImage.start()
await latiss.rem.atcamera.cmd_disableCalibration.start()


groupId: 2021-10-07T18:48:56.278,imageType: ENGTEST


Step 2C:
The following commands should take a calibration image\
with multiple(4) discardRows()\
All in one cell so we get a real 2 second image

In [29]:
await latiss.rem.atcamera.cmd_enableCalibration.set_start()
await latiss.rem.atcamera.cmd_clear.set_start(nClears=2)
keyValueMap = f"groupId: {Time.now().isot},imageType: ENGTEST"
print(keyValueMap)
latiss.rem.atcamera.cmd_startImage.set(shutter=True, keyValueMap=keyValueMap, timeout=10.0)
await latiss.rem.atcamera.cmd_startImage.start()
for i in range(4):
    await asyncio.sleep(0.5)
    latiss.rem.atcamera.cmd_discardRows.set(nRows=500)
    await latiss.rem.atcamera.cmd_discardRows.start()
await latiss.rem.atcamera.cmd_endImage.start()
await latiss.rem.atcamera.cmd_disableCalibration.start()


groupId: 2021-10-07T18:52:28.209,imageType: ENGTEST


In [30]:
# Take a bias to make sure everything is working
await latiss.take_bias(1)

Generating group_id
imagetype: BIAS, skip TCS synchronization.
BIAS 0001 - 0001


array([2021100700125])

In [31]:
# Putting everything back in standby.
await latiss.standby()

Unable to transition atcamera to <State.STANDBY: 5> NoneType: None
.
Traceback (most recent call last):
  File "/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-0.7.0/lib/python3.8/site-packages/lsst/ts/salobj/csc_utils.py", line 161, in set_summary_state
    await cmd.start(timeout=timeout)
  File "/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-0.7.0/lib/python3.8/site-packages/lsst/ts/salobj/topics/remote_command.py", line 483, in start
    return await cmd_info.next_ackcmd(timeout=timeout)
  File "/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-0.7.0/lib/python3.8/site-packages/lsst/ts/salobj/topics/remote_command.py", line 201, in next_ackcmd
    raise base.AckError(msg="Command failed", ackcmd=ackcmd)
lsst.ts.salobj.base.AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=1659503270, ack=<SalRetCode.CMD_FAILED: -302>, error=0, result='Error : java.util.concurrent.TimeoutException: Timed out after 1000 m

RuntimeError: Failed to transition ['atcamera'] to <State.STANDBY: 5>.

Stuff below is just in case it is needed.

In [ ]:
await latiss.rem.atcamera.cmd_disable.set_start()

In [ ]:
latiss.rem.atcamera.cmd_endImage.set_start()

In [ ]:
await latiss.rem.atcamera.cmd_disableCalibration.start()

In [ ]:
await salobj.set_summary_state(latiss.rem.atcamera, salobj.State.ENABLED)

In [ ]:
test = await latiss.rem.atcamera.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
print(test)

In [ ]:
print(test)

In [ ]:
await latiss.rem.atcamera.cmd_disable?

In [ ]:
await latiss.rem.atcamera.cmd_disable

In [ ]:
await latiss.rem.atcamera.cmd_disable.set_start()

In [ ]:
await salobj.set_summary_state(latiss.rem.atspectrograph, salobj.State.STANDBY)

In [ ]:
await latiss.rem.atspectrograph.cmd_enable.set_start({"atspectrograph": "current"})

In [ ]:
await latiss.rem.atspectrograph.enable({"atspectrograph": "current"})

In [ ]:
await latiss.enable({"atspectrograph": "current"})

In [ ]:
await latiss.rem.atcamera.cmd_initImage.set_start(deltaT=2.0)

In [ ]:
await latiss.rem.atcamera.cmd_abort.start()

In [ ]:
await latiss.rem.atcamera.cmd_stop.start()

In [ ]:
await latiss.rem.atcamera.cmd_stop

In [ ]:
await latiss.rem.atcamera.cmd_endImage.set?

In [ ]:
await latiss.rem.atcamera.cmd_endImage.set

In [ ]:
await latiss.rem.atcamera.cmd_endImage